# Process Data
* Varous data cleaning tasks

In [ ]:
import os

import papermill as pm

In [ ]:
data_path = "../../data/papermill/process_data"
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

In [ ]:
def run_notebook(script, name=None, parameters=None):
    if name is None:
        name = script
    parameters = parameters if parameters is not None else dict()
    pm.execute_notebook(
        script,
        os.path.join(data_path, name),
        parameters=parameters,
    )

In [ ]:
def run_notebooks(script):
    for media in ["manga", "anime"]:
        run_notebook(
            f"{script}.ipynb", f"{script}{media.capitalize()}.ipynb", {"media": media}
        )

In [ ]:
run_notebooks("ProcessMedia")

In [ ]:
run_notebooks("ProcessMediaLists")

In [ ]:
run_notebook("PruneMediaLists.ipynb")

In [ ]:
run_notebook("RelabelIds.ipynb")

In [ ]:
run_notebooks("RelatedMediaGraph")

In [ ]:
run_notebooks("RelatedCrossMediaGraph")

In [ ]:
run_notebooks("GenerateSplits")

In [ ]:
run_notebooks("CompressSplits")

In [ ]:
print("Success!")